In [2]:
import pyautogui, pynput, pandas, numpy, datetime, matplotlib.pyplot, json
from time import sleep

In [9]:
class Class:

    def __init__(self):

        self.pressed = False

    def listen(self, key):

        if (key == pynput.keyboard.Key.ctrl_l): self.pressed = True

    def checkpoint(self, string: list[str]):

        print(string)
        with pynput.keyboard.Listener(on_press = self.listen) as listener:
            while not self.pressed: pass
        listener.join()

In [13]:
with open("generator.json", "r") as file:
    generator = json.load(file)

symbols, meshes, params = generator.values()
params = pandas.DataFrame(params).transpose()
template = pandas.DataFrame(columns = params.index)
backtests = template.copy()
for mesh in meshes:
    values = params.loc[mesh, "values"]
    values = numpy.meshgrid(*values)
    for n, dim in enumerate(values):
        values[n] = dim.reshape(-1)
    values = numpy.stack(values)
    backtests_mesh = template.copy()
    backtests_mesh[mesh] = values.T
    fixed = list(set(params.index).difference(mesh))
    backtests_mesh[fixed] = params.loc[fixed, "def"]
    backtests = pandas.concat((backtests, backtests_mesh))

for parameter in backtests.columns:
    values = params.at[parameter, "values"]
    backtests_mesh = template.copy()
    backtests_mesh[parameter] = values
    fixed = list(backtests.columns)
    fixed.remove(parameter)
    backtests_mesh[fixed] = params.loc[fixed, "def"]
    backtests = pandas.concat((backtests, backtests_mesh))

backtests = backtests.drop_duplicates()
backtests = backtests.sort_values(list(backtests.columns))
backtests = backtests.reset_index(drop = True)
backtests.to_csv("configs.csv", index = False)

namer = ", ".join(["%s = {0[%s]}" % (column, column) for column in backtests.columns])
backtests["report"] = backtests.agg(namer.format, axis = "columns") + ".csv"
replace = {":": ".", "/": "|", "\\": "|"}
for a, b in replace.items():
    backtests["report"] = backtests["report"].str.replace(a, b)

Backtests = pandas.DataFrame()
symbols = [
    "EURUSD", "GBPUSD", "USDCHF", "USDCAD",
    "EURAUD", "GBPJPY", "EURCAD", "EURGBP"
]
for symbol in symbols:
    backtests["symbol"] = symbol
    backtests["EA"] = "Forex Fury v4"
    Backtests = pandas.concat((Backtests, backtests), axis = "index")

Backtests["report"] = "EA = " + Backtests["EA"] + ", symbol = " + Backtests["symbol"] + ", " + Backtests["report"]
Backtests = Backtests[["symbol", "EA", "SL", "TP", "maxSpread", "ts", "tf", "report"]]
for symbol in symbols:
    Backtests.loc[Backtests["symbol"] == symbol].to_csv(symbol + ".csv", index = False)

C:\Users\Oreo\AppData\Local\Temp/ipykernel_26260/417168211.py:38: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  backtests["report"] = backtests["report"].str.replace(a, b)
